In [3]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
from heapq import merge
from sklearn import preprocessing
import gc
import os

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

from src

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

In [35]:
rand.seed(37)
AAMass = {'A':71.037114, 'C':103.009185, 'D':115.026943, 'E':129.042593, 'F':147.068414, 'G':57.021464, 'H':137.058912,
          'I':113.084064, 'K':128.094963, 'L':113.084064, 'M':131.040485, 'N':114.042927, 'P':97.052764, 'Q':128.058578,
          'R':156.101111, 'S':87.032028, 'T':101.047679, 'V':99.068414, 'W':186.079313, 'Y':163.0633}

H2O = 18.015
NH3 = 17.031
PROTON = 1.00727647
specsize = 8000
charge = 2
use_mods = False

In [36]:
def GetAAMass(AA):
    return AAMass[AA] + 57.021464 if AA == 'C' else AAMass[AA]

def GetSpectrum(Seq):
    size = len(Seq)
    outsize = 2*size
    bspectrum = []
    bspectrumaux = []
    yspectrum = []
    yspectrumaux = []
    
    bspectrum.append(GetAAMass(Seq[0]) + PROTON)
    yspectrum.append(GetAAMass(Seq[-1]) + H2O + PROTON)
    
    for i, (fAA, bAA) in enumerate(zip((Seq[1:]), Seq[-2::-1])):
        bspectrum.append(bspectrum[i] + GetAAMass(fAA))
        yspectrum.append(yspectrum[i] + GetAAMass(bAA))
    
#    bspectrumaux = [(bion + PROTON) / 2 for bion in bspectrum]
#    bspectrumaux.extend([bion - H2O for bion in bspectrum])
#    bspectrumaux.extend([bion - NH3 for bion in bspectrum])
    
#    yspectrumaux = [(yion + PROTON) / 2 for yion in yspectrum]
#    yspectrumaux.extend([yion - H2O for yion in yspectrum])
#    yspectrumaux.extend([yion - NH3 for yion in yspectrum])
    
#    bspectrum.extend(bspectrumaux)
#    yspectrum.extend(yspectrumaux)
    
#     bspectrum.sort()
#     yspectrum.sort()
    
    mergedout = list(merge(bspectrum, yspectrum))
    if mergedout[-1] > specsize:
        print(mergedout[-1])
        print(Seq)
    tspec = np.zeros(specsize)
    tspec[np.rint(mergedout).astype(int)] = 1
    return tspec

by = GetSpectrum('ACDEFG')
print('printing by:\n' + str(by))
print(by.shape)
print(AAMass['L'])

printing by:
[0. 0. 0. ... 0. 0. 0.]
(8000,)
113.084064


In [37]:
def reset_signals():
    isName = isMW = isNumPeaks = False

In [38]:
f = open("data/human_consensus_final_true_lib.msp", "r")
lines = f.readlines()
newcontents = []
specid = 0
prev = 0
num_specs = 0
for i, line in enumerate(lines):
    if line.startswith('Name') and '_' in line:
        l_charge = int(line[line.find('_') - 1])
        if l_charge != charge:
            continue
        if use_mods:
            num_specs += 1
        elif '(' not in line and ')' not in line:
            num_specs += 1
        new = int((i/len(lines)) * 100)
        if (new > prev):
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new
f.close()
print(type(lines))
print('len of file: ' + str(len(lines)))
print(num_specs)

99%
<class 'list'>
len of file: 50017570
130172


#### Sin Mods:
Number of charge 1 examples: 18230  
Number of charge 2 examples: 130172  
Number of charge 3 examples: 70741  
  
#### Con Mods:
Number of charge 2 examples: 184994

In [39]:
def getrandmod(seq, nummods=1):
    AAs = list(AAMass.keys())
    res = temp = seq
    for i in range(nummods):
        while res == temp:
            randindx = rand.randint(0, len(seq)-1)
            randmod = AAs[rand.randint(0, len(AAs))-1]
            temp = temp[:randindx] + randmod + temp[randindx+1:]
        res = temp
    return res

In [40]:
seq = 'AFINSTWDG'
print(getrandmod(seq))

AWINSTWDG


In [53]:
def readmspwithdecoy(mspfile):
    f=open(mspfile, "r")
    lines = f.readlines()
    f.close()
    #fo = open('output.csv', 'w')
    #fo.write('Q,P,N\n')
    fixedlen = 300
    dataset = []
    label = []
    print('len of file: ' + str(len(lines)))
    count = 0
    limit = 200000
    pep = mass = numPeaks = 0
    #spec = []
    isName = isMW = isNumPeaks = False
    new = prev = 0
    maxpeaks = maxmoz = 0
    i = 0
    while i < len(lines) and limit > 0:
        line = lines[i]
        i += 1
        splits = line.split(':') 
        if (splits[0] == 'Name') and '_' in line:
            split1 = splits[1]
            l_charge = int(split1[split1.find('_') - 1])
            if l_charge != charge: # l_charge == l_charge always true.
                continue
            if use_mods:
                pep = split1.split('/')[0].lstrip(' ')
                isName = True
            elif '(' not in splits[1] and ')' not in splits[1]:
                pep = split1.split('/')[0].lstrip(' ')
                isName = True

        if (isName and splits[0] == 'MW'):
            mass = float(splits[1])
            if round(mass) < specsize:
                isMW = True
                #limit = limit - 1
            else:
                isName = isMW = isNumPeaks = False
                continue

        if (isName and isMW and splits[0] == 'Num peaks'):
            numPeaks = int(splits[1])
            if numPeaks > maxpeaks:
                maxpeaks = numPeaks

            spec = np.zeros(specsize)
            while (lines[i] != '\n'):
                mzline = lines[i]
                i +=1
                mzsplits = mzline.split('\t')
                moz, intensity = float(mzsplits[0]), float(mzsplits[1])
                if moz > maxmoz:
                    maxmoz = moz
                spec[round(moz)] += round(intensity)

            spec = np.clip(spec, None, 1000.0)
            spec = preprocessing.scale(spec)

            isNumPeaks = True

        if isName and isMW and isNumPeaks:
            isName = isMW = isNumPeaks = False
            #revPep = pep[0] + pep[1:-1][::-1] + pep[-1]
            revPep = getrandmod(pep)
            if pep == revPep:
                print('decoy is the same. shuffling')
                #revPep = ''.join(rand.sample(revPep,len(revPep)))
                revPep = getrandmod(pep, len(pep))
                print(pep)
                print(revPep)
            tspec = preprocessing.scale(GetSpectrum(pep))
            rtspec = preprocessing.scale(GetSpectrum(revPep))

            dataset.append([spec, tspec, rtspec])
            label.append([1, -1])

            count = count + 1
            pep = mass = numPeaks = 0
            spec = []
            new = int((i/len(lines)) * 100)
            if (new > prev):
                #clear_output(wait=True)
                print(str(new) + '%')
                prev = new
    
    print('max peaks: ' + str(maxpeaks))
    print('count: ' + str(count))
    print('max moz: ' + str(maxmoz))
    return dataset, label
# print('max peaks: ' + str(maxpeaks))
# print('count: ' + str(count))
# print('max moz: ' + str(maxmoz))
# #fo.close()

In [54]:
dataset, label = readmspwithdecoy("data/human_consensus_final_true_lib.msp")

len of file: 50017570
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
max peaks: 251
count: 130172
max moz: 2616.7


In [ ]:
gc.collect()

In [55]:
print('splitting...')
tmpTrainData, tmpTestData = train_test_split(
            dataset, test_size = 0.2, random_state = rand.randint(0, 1000), shuffle = True)

print('converting to tensors...')
XTrainTensor = torch.tensor(tmpTrainData, dtype=torch.float)
XTestTensor = torch.tensor(tmpTestData, dtype=torch.float)
print('done')
del dataset, tmpTrainData, tmpTestData

splitting...
converting to tensors...
done


In [ ]:
del tmpTrainData, tmpTestData

In [56]:
batch_size = 1024

train_loader = torch.utils.data.DataLoader(dataset=XTrainTensor, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=XTestTensor, batch_size=batch_size, shuffle=False)
del XTrainTensor, XTestTensor

In [57]:
do_learn = True
save_frequency = 2
lr = 0.00001
num_epochs = 200
weight_decay = 0.0001
margin = 0.2
#torch.manual_seed(0)
#torch.cuda.manual_seed(0)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

In [58]:
#train_loader = torch.load('train_loader.pt')
#test_loader = torch.load('test_loader.pt')

In [59]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.searching = False
        
        self.linear1_1 = nn.Linear(specsize, 1024)
        self.linear1_2 = nn.Linear(1024, 512)
        #self.linear1_3 = nn.Linear(512, 256)
        
        self.linear2_1 = nn.Linear(specsize, 1024)
        self.linear2_2 = nn.Linear(1024, 512)
        #self.linear2_3 = nn.Linear(512, 256)
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.3)
        #self.dropout3 = nn.Dropout(0.3)
        
    def forward(self, data):
        res = []
        x = data[:, 0]
        x = self.linear1_1(x.view(-1, specsize))
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.linear1_2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        #x = self.dropout2(x)
        #x = self.linear1_3(x)
        #x = F.relu(x)
        #if not self.searching:
        #    x = F.normalize(x)
        res.append(x)
        for i in range(data.shape[1]-1):
            x = data[:, i+1]
            x = self.linear2_1(x.view(-1, specsize))
            x = F.relu(x)
            x = self.dropout1(x)
            x = self.linear2_2(x)
            x = F.relu(x)
            x = self.dropout2(x)
            #x = self.dropout2(x)
            #x = self.linear2_3(x)
            #x = F.relu(x)
            #if not self.searching:
            #x = F.normalize(x)
            #x = self.linear1_3(x)
            #x = F.relu(x)
            res.append(x)
        
        return res
    
    def name(self):
        return "Net"

In [60]:
for (tid, data) in enumerate(train_loader):
    if tid == 0:
        print(data[:,0:3:2].shape)
        break

torch.Size([1024, 2, 8000])


In [61]:
#hinge = torch.nn.HingeEmbeddingLoss()
triplet_loss = nn.TripletMarginLoss(margin=margin, p=2, reduction='sum')

def train(model, device, train_loader, epoch, optimizer):
    model.train()
    
    accurate_labels = 0
    all_labels = 0
    for (batch_idx, data) in enumerate(train_loader):
        
        data = data.to(device)    
        
        optimizer.zero_grad()
        
        Q, P, N = model(data)
        
        loss = triplet_loss(Q, P, N)
        
        loss.backward()
            
        optimizer.step()
        
        accurate_labels = accurate_labels + 2*torch.sum(F.pairwise_distance(Q, P)**2 - 
                                                        F.pairwise_distance(Q, N)**2 + margin < 0)
        
        all_labels = all_labels + 2*len(Q)  
    
    accuracy = 100. * float(accurate_labels) / all_labels
    train_accuracy.append(accuracy)
    train_loss.append(loss/batch_size)
    print('Epoch: ' + str(epoch))
    print('Train accuracy: {}/{} ({:.3f}%)\tLoss: {:.6f}'.format(accurate_labels, all_labels, accuracy, loss))
    
def test(model, device, test_loader):
    model.eval()
    
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        
        for (batch_idx, data) in enumerate(test_loader):
            
            data = data.to(device)    
        
            optimizer.zero_grad()
            
            Q, P, N = model(data)
            
            loss = triplet_loss(Q, P, N)
            
            accurate_labels = accurate_labels + 2*torch.sum(F.pairwise_distance(Q, P)**2 - 
                                                        F.pairwise_distance(Q, N)**2 + margin < 0)
            
            all_labels = all_labels + 2*len(Q)
                
        accuracy = 100. * float(accurate_labels) / all_labels
        test_accuracy.append(accuracy)
        test_loss.append(loss/batch_size)
        print('Test accuracy: {}/{} ({:.3f}%)\tLoss: {:.6f}'.format(accurate_labels, all_labels, accuracy, loss))

In [62]:
def oneshot(model, device, data):
    model.eval()
    
    with torch.no_grad():
        for i in range(len(data)):
            data[i] = data[i].to(device)
            
        output = model(data)
        return torch.squeeze(torch.argmax(output, dim=1)).cpu().item()

In [63]:
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#model = Net().to(device)
model.linear1_1.weight.requires_grad = False
model.linear1_1.bias.requires_grad = False
model.linear1_2.weight.requires_grad = False
model.linear1_2.bias.requires_grad = False
   
if do_learn: # training mode
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    #optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        train(model, device, train_loader, epoch, optimizer)
        test(model, device, test_loader)
         

Epoch: 0
Train accuracy: 146106/208274 (70.151%)	Loss: 91.559074
Test accuracy: 44192/52070 (84.870%)	Loss: 28.814447
Epoch: 1
Train accuracy: 145664/208274 (69.939%)	Loss: 93.563744
Test accuracy: 43314/52070 (83.184%)	Loss: 30.347076
Epoch: 2
Train accuracy: 144826/208274 (69.536%)	Loss: 97.943008
Test accuracy: 41974/52070 (80.611%)	Loss: 32.558372
Epoch: 3
Train accuracy: 143316/208274 (68.811%)	Loss: 93.328125
Test accuracy: 40336/52070 (77.465%)	Loss: 34.893486
Epoch: 4
Train accuracy: 141330/208274 (67.858%)	Loss: 85.903488
Test accuracy: 37994/52070 (72.967%)	Loss: 37.723900
Epoch: 5
Train accuracy: 140060/208274 (67.248%)	Loss: 97.854370
Test accuracy: 35352/52070 (67.893%)	Loss: 40.553528
Epoch: 6
Train accuracy: 138700/208274 (66.595%)	Loss: 89.202606
Test accuracy: 32178/52070 (61.798%)	Loss: 43.620953
Epoch: 7
Train accuracy: 137170/208274 (65.860%)	Loss: 89.111954
Test accuracy: 28792/52070 (55.295%)	Loss: 47.016983


KeyboardInterrupt: 

In [ ]:
torch.save(model, 'models/siamese99.7%.pt')

remove modifications  
use one charge